# Challenges

In this episode, we will provide two different challenges for you to attempt, based on the skills you have acquired so far. One of the challenges will be related to the shape of objects in images (morphometrics), while the other will be related to colors of objects in images (colorimetrics). We will not provide solution code for either of the challenges, but your instructors should be able to give you some gentle hints if you need them.

------


### Bacteria Colony Counting

As mentioned in the workshop _introduction_, 
your first final challenge is to determine how many bacteria colonies are in 
each of these images. As you already know, these images can be found in the 
**/data/** directory of the repository. 

![Colony image 1](../fig/00-colonies01.jpg)

![Colony image 2](../fig/00-colonies02.jpg)

![Colony image 3](../fig/00-colonies03.jpg)

Write a Python program that uses skimage to count the number of bacteria colonies in each image, and for each, produce a new image that labels the colonies. Your output should be similar to this image:

![colonies01 result](../fig/10-colonies01_overlay.png)

This time you won't get detailed instructions for every step. Instead, try to come up with a solution strategy yourself.

_Hint: You might want to use most of the analysis operations you already learned during the workshop, it's OK to copy code from preceding episodes!_

**Bonus Task:** Generate a histogram plot, which shows the colony size distributions. 

_Hint: This is a good opportunity to use the convenience Matplotlib `plt.hist()` function, [read the docs here](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist.html)._ 

### Colorimetrics: titration color analysis

The video showing the titration process first mentioned in the workshop 
_introduction_ episode can be found in the 
**/data/titration-movie** directory.
Write a Python program that uses skimage to analyze the video on a
frame-by-frame basis. Your program should do the following:

1. Sample a ROI from the same location on each frame, and determine the 
	average red, green, and blue channel value.

2. Display a graph plotting the average color channel values as a function of
	the frame number, similar to this image:

	![Titration colors](../fig/00-colorimetric-result.png)

3. Save the graph as an image named **titration.png**.

4. Output a CSV file named **titration.csv**, with each line containing
	the frame number, average red value, average green value, and average
	blue value
    
_Hint: The movie is given as an image sequence in `/data/titration-movie/image*.jpg`. Use `sorted( glob.glob(...) )` to crawl through the directory and read the images with `skimage.io`. Using a Pandas `DataFrame` to record the measurements might be a good idea, but no must._
